# Lesson Overview 

1. Review of TF Transform (TFT)

2. Dataset Review & Define Raw Features

3. Run steps from previous TF Data Validation pipeline

4. Pre-Process with tf.Transform

5. Transform the data using Apache Beam Pipeline

6. Create an Input Function for the Training

7. Create an Input Function for Serving

8. Define an Estimator 

9. Train, Evaluate, and Export our model

## TensorFlow Transform

#### The Feature Engineering Component of TensorFlow Extended (TFX)

[TensorFlow Transform](https://github.com/tensorflow/transform) is a library for preprocessing data with TensorFlow. `tf.Transform` is useful for data that requires a full-pass, such as:

+ Normalize an input value by mean and standard deviation.

+ Convert strings to integers by generating a vocabulary over all input values.

+ Convert floats to integers by assigning them to buckets based on the observed data distribution.

+ TensorFlow has built-in support for manipulations on a single example or a batch of examples. `tf.Transform` extends these capabilities to support full-passes over the example data.

We will export the output of `tf.Transform` as a TensorFlow graph to use for training and serving. Using the same graph for both training and serving can prevent skew since the same transformations are applied in both stages.

## Dataset

The dataset will be using throughout this session will be the New York Yellow Cab dataset available via [BigQuery public datasets](bigquery-public-data:new_york.tlc_yellow_trips_2016).

The columns in the dataset are:

![title](../assets/schema.png)

## Install necessary packages

You will want to install the necessary packages. Run the following command `pip install -r requirements.txt`.
At the moment, Apache Beam (2.8.0) is only compatible with Python 2.7, however; a Python 3 version should be available soon.

In [1]:
# check if you are using Python 2
import sys
assert sys.version_info.major is 2, 'Oops, not running Python 2'

In [ ]:
!pip2 install -r ../requirements.txt

# >>> Please remember to restart the kernel before you continue so the recently installed packages above get loaded into your sesion. <<<

To restart the Kernel, click on "Kernel" from the top menu & then "Restart".

## Load necessary packages

In [2]:
import warnings
warnings.filterwarnings("ignore")

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import shutil
import argparse
import tensorflow as tf
import apache_beam as beam  
import tensorflow_transform as tft
from google.protobuf import text_format 
import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv
import tensorflow_transform.beam as tft_beam
from tensorflow.python.lib.io import file_io
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_transform import coders as tft_coders
from tensorflow_transform.tf_metadata import metadata_io
from tensorflow_transform.saved import saved_transform_io
from tensorflow_transform.beam.tft_beam_io import transform_fn_io
from tensorflow_transform.coders import example_proto_coder
from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import schema_utils

/Users/arm/anaconda3/envs/py27_oreilly_ml_tfx_course/lib/python2.7/site-packages/scipy/optimize/_minimize.py:37: ImportWarning: Not importing directory '/Users/arm/anaconda3/envs/py27_oreilly_ml_tfx_course/lib/python2.7/site-packages/scipy/optimize/lbfgsb': missing __init__.py
  from .lbfgsb import _minimize_lbfgsb


## Define Lesson-wide Parameters

In [3]:
BASE_DIR = os.getcwd()

DATA_DIR = os.path.join(BASE_DIR, '../data')

OUTPUT_DIR = os.path.join(BASE_DIR)

# base dir containing train and eval data
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')
EVAL_DATA_DIR = os.path.join(DATA_DIR, 'eval')
SERVING_DATA_DIR = os.path.join(DATA_DIR, 'serving')

TRAIN_DATA = os.path.join(TRAIN_DATA_DIR, 'train.csv')
EVAL_DATA = os.path.join(EVAL_DATA_DIR, 'eval.csv')
SERVING_DATA = os.path.join(SERVING_DATA_DIR, 'serving.csv')

# base dir where TFT writes training data
TFT_TRAIN_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_train')
TFT_TRAIN_FILE_PREFIX = 'train_transformed'

# base dir where TFT writes eval data
TFT_EVAL_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tft_eval')
TFT_EVAL_FILE_PREFIX = 'eval_transformed'

TF_OUTPUT_BASE_DIR = os.path.join(OUTPUT_DIR, 'tf')
 
SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'


def get_tft_train_output_dir(run_id):
    return _get_output_dir(TFT_TRAIN_OUTPUT_BASE_DIR, run_id)

def get_tft_eval_output_dir(run_id):
    return _get_output_dir(TFT_EVAL_OUTPUT_BASE_DIR, run_id)

def get_tf_output_dir(run_id):
    return _get_output_dir(TF_OUTPUT_BASE_DIR, run_id)

def _get_output_dir(base_dir, run_id):
    return os.path.join(base_dir, 'run_' + str(run_id))

def get_schema_file():
    return os.path.join(OUTPUT_DIR, 'schema.pbtxt')

## Remove output from previous runs

In [4]:
shutil.rmtree(TFT_TRAIN_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TFT_EVAL_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(TF_OUTPUT_BASE_DIR, ignore_errors=True)
shutil.rmtree(get_schema_file(), ignore_errors=True)

## Define Raw Features

In [5]:
# categorical features are assumed to each have a maximum value in the dataset.
MAX_CATEGORICAL_FEATURE_VALUES = [2]

CATEGORICAL_FEATURE_KEYS = []

DENSE_FLOAT_FEATURE_KEYS = ['trip_distance', 'passenger_count', 'tip_amount']

# number of buckets used by tf.transform for encoding each feature.
FEATURE_BUCKET_COUNT = 10

BUCKET_FEATURE_KEYS = ['pickup_hour', 
                       'pickup_month', 
                       'pickup_day_of_week', 
                       'dropoff_month',
                       'dropoff_hour',
                       'dropoff_day_of_week']

# number of vocabulary terms used for encoding VOCAB_FEATURES by tf.transform
VOCAB_SIZE = 1000

# count of out-of-vocab buckets in which unrecognized VOCAB_FEATURES are hashed.
OOV_SIZE = 10

VOCAB_FEATURE_KEYS = []

LABEL_KEY = 'fare_amount'

CSV_COLUMN_NAMES = [
    'vendor_id',
    'pickup_month',
    'pickup_hour',
    'pickup_day_of_week',
    'dropoff_month',
    'dropoff_hour',
    'dropoff_day_of_week',
    'passenger_count',
    'trip_distance',
    'fare_amount',
    'tip_amount',
    'payment_type',
    'trip_type',]

## Define Helper Methods

In [6]:
def transformed_name(key):
    return key + '_xf'

def transformed_names(keys):
    return [transformed_name(key) for key in keys]

# Tf.Transform considers these features as "raw"
def get_raw_feature_spec(schema):
    return schema_utils.schema_as_feature_spec(schema).feature_spec

def make_proto_coder(schema):
    raw_feature_spec = get_raw_feature_spec(schema)
    raw_schema = dataset_schema.from_feature_spec(raw_feature_spec)
    return tft_coders.ExampleProtoCoder(raw_schema)

def make_csv_coder(schema):
    """Return a coder for tf.transform to read csv files."""
    raw_feature_spec = get_raw_feature_spec(schema)
    parsing_schema = dataset_schema.from_feature_spec(raw_feature_spec)
    return tft_coders.CsvCoder(CSV_COLUMN_NAMES, parsing_schema)

def clean_raw_data_dict(input_dict, raw_feature_spec):
    """Clean raw data dict."""
    output_dict = {}

    for key in raw_feature_spec:
        if key not in input_dict or not input_dict[key]:
            output_dict[key] = []
        else:
            output_dict[key] = [input_dict[key]]
    return output_dict

def read_schema(path):
    """Reads a schema from the provided location.

    Args:
    path: The location of the file holding a serialized Schema proto.

    Returns:
    An instance of Schema or None if the input argument is None
    """
    result = schema_pb2.Schema()
    contents = file_io.read_file_to_string(path)
    text_format.Parse(contents, result)
    return result

## Run steps from previous TF Data Validation pipeline

In [7]:
train_stats = tfdv.generate_statistics_from_csv(data_location=os.path.join(TRAIN_DATA_DIR, 'train.csv'))
schema = tfdv.infer_schema(statistics=train_stats, infer_feature_shape=False)

# persist schema
file_io.recursive_create_dir(OUTPUT_DIR)
file_io.write_string_to_file(get_schema_file(), text_format.MessageToString(schema))

## Pre-process with tf.Transform

The pre-process function is where the transformations of the dataset occur using `tf.Transform`. Function accepts and returns a dictionary of tensors, where a tensor means a Tensor or SparseTensor. 

The implementation of the pre-process function consist of the two main groups of API calls:

1. **TensorFlow Ops:** Any function that accepts and returns tensors (meaning TensorFlow ops). These add TensorFlow operations to the graph that transforms raw data into transformed data one feature vector at a time. These will run for every example, during both training and serving.

2. **TensorFlow Transform Analyzers:** Any of the analyzers provided by `tf.Transform`. **Analyzers** also accept and return tensors, but unlike TensorFlow ops they **only run once**, during training, and typically make a full pass over the entire training dataset. They create tensor constants, which are added to your graph. 

For example, `tft.min` computes the minimum of a tensor over the training dataset. `tf.Transform` provides a fixed set of [analyzers](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/analyzers.py) which will likely be extended in the future.

## Define Pre-Process function

In [8]:
def preprocessing(inputs):
    """tf.transform's callback function for preprocess inputs.

    Args:
      inputs: map from feature keys to raw not-yet-transformed features.

    Returns:
      Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in DENSE_FLOAT_FEATURE_KEYS:
      # Preserve this feature as a dense float, setting nan's to the mean.
      outputs[transformed_name(key)] = tft.scale_to_z_score(
          _fill_in_missing(inputs[key]))

    for key in VOCAB_FEATURE_KEYS:
      # Build a vocabulary for this feature.
      outputs[
          transformed_name(key)] = tft.compute_and_apply_vocabulary(
              _fill_in_missing(inputs[key]),
              top_k=VOCAB_SIZE,
              num_oov_buckets=OOV_SIZE)

    for key in BUCKET_FEATURE_KEYS:
        outputs[transformed_name(key)] = tft.bucketize(
          _fill_in_missing(inputs[key]), FEATURE_BUCKET_COUNT)

    for key in CATEGORICAL_FEATURE_KEYS:
        outputs[transformed_name(key)] = _fill_in_missing(inputs[key])

    fare_amount = _fill_in_missing(inputs[LABEL_KEY])

    outputs[transformed_name(LABEL_KEY)] = fare_amount

    return outputs

In [9]:
def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.

    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.

    Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.

    Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
    """
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(tf.sparse_to_dense(x.indices, [x.dense_shape[0], 1], 
                      x.values,
                      default_value),
                      axis=1)

In [10]:
def transform_data(input_handle,
                   outfile_prefix,
                   working_dir,
                   schema_file,
                   transform_dir=None,
                   max_rows=None,
                   pipeline_args=None):
    """The main tf.transform method which analyzes and transforms data.

    Args:
    input_handle: Path to csv file with input data.
    outfile_prefix: Filename prefix for emitted transformed examples
    working_dir: Directory in which transformed examples and transform function
      will be emitted.
    schema_file: An file path that contains a text-serialized TensorFlow
      metadata schema of the input data.
    transform_dir: Directory in which the transform output is located. If
      provided, this will load the transform_fn from disk instead of computing
      it over the data. Hint: this is useful for transforming eval data.
    max_rows: Number of rows to query from BigQuery
    pipeline_args: additional DataflowRunner or DirectRunner args passed to the
      beam pipeline.
    """

    schema = read_schema(schema_file)
    raw_feature_spec = get_raw_feature_spec(schema)
    raw_schema = dataset_schema.from_feature_spec(raw_feature_spec)
    raw_data_metadata = dataset_metadata.DatasetMetadata(raw_schema)

    with beam.Pipeline(argv=pipeline_args) as pipeline:
        with tft_beam.Context(temp_dir=working_dir):
            if input_handle.lower().endswith('csv'):
                # read raw train data from csv file
                csv_coder = make_csv_coder(schema)
                raw_data = (
                    pipeline
                    | 'ReadFromText' >> beam.io.ReadFromText(
                        input_handle, skip_header_lines=1)
                    | 'ParseCSV' >> beam.Map(csv_coder.decode))
            if transform_dir is None:
                # analyze and transform raw training data to produced transform_fn
                transform_fn = (
                    (raw_data, raw_data_metadata)
                    | ('Analyze' >> tft_beam.AnalyzeDataset(preprocessing)))
                # write transform_fn as tf.graph
                _ = (
                    transform_fn
                    | ('WriteTransformFn' >>
                       tft_beam.WriteTransformFn(working_dir)))
            else:
                transform_fn = pipeline | tft_beam.ReadTransformFn(transform_dir)

            # shuffling the data before to improve training
            shuffled_data = raw_data | 'RandomizeData' >> beam.transforms.Reshuffle()

            # get data and schema separately from the raw_data_metadata
            (transformed_data, transformed_metadata) = (
              ((shuffled_data, raw_data_metadata), transform_fn)
              | 'Transform' >> tft_beam.TransformDataset())

            # write transformed train data to sink
            coder = example_proto_coder.ExampleProtoCoder(transformed_metadata.schema)
            _ = (
              transformed_data
              | 'SerializeExamples' >> beam.Map(coder.encode)
              | 'WriteExamples' >> beam.io.WriteToTFRecord(
                  os.path.join(working_dir, outfile_prefix), file_name_suffix='.gz')
            )

## Transform the data using Apache Beam Pipeline

Here are the steps of our pipeline:

    1) Read in the data using the CSV reader

    2) Transform raw training data using a preprocessing pipeline that scales numeric data, replaces missing values & bucketize feature values.

    3) Shuffling the data before to improve training

    4) Write out the result as a TFRecord

## Using our Pre-processed data to train a model

`tf.Transform` helps us prevent train/serve skew by using the same code for both training and serving.

In order to prevent train/serve skew, we need to train a model & prepare our trained model for production by creating input functions. The main difference between our training input function and our serving input function is that training data contains the labels, and production data does not.  

## Create an Input Function for the Training

In [11]:
def _gzip_reader_fn():
    """Small utility returning a record reader that can read gzip'ed files."""
    return tf.TFRecordReader(
      options=tf.python_io.TFRecordOptions(compression_type=tf.python_io.TFRecordCompressionType.GZIP))

In [12]:
def input_fn(filenames, tf_transform_dir, batch_size=200):
    """Generates features and labels for training or evaluation.

    Args:
    filenames: [str] list of CSV files to read data from.
    tf_transform_dir: directory in which the tf-transform model was written
      during the preprocessing step.
    batch_size: int First dimension size of the Tensors returned by input_fn

    Returns:
    A (features, indices) tuple where features is a dictionary of
      Tensors, and indices is a single Tensor of label indices.
    """
    metadata_dir = os.path.join(tf_transform_dir,
                              transform_fn_io.TRANSFORMED_METADATA_DIR)
    transformed_metadata = metadata_io.read_metadata(metadata_dir)
    transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

    transformed_features = tf.contrib.learn.io.read_batch_features(
      filenames, batch_size, transformed_feature_spec, reader=_gzip_reader_fn)

    # we pop the label because we do not want to use it as a feature while we're
    # training.
    return transformed_features, transformed_features.pop(
      transformed_name(LABEL_KEY))

## Create an Input Function for Serving

Let's create an input function that we could use in production, and prepare our trained model for serving.

In [13]:
SERVING_MODEL_DIR = 'serving_model_dir'
EVAL_MODEL_DIR = 'eval_model_dir'

TRAIN_BATCH_SIZE = 40
EVAL_BATCH_SIZE = 40

FIRST_DNN_LAYER_SIZE = 100
NUM_DNN_LAYERS = 4
DNN_DECAY_FACTOR = 0.7

## Define an Estimator 

Let's wrap our input data in FeatureColumns...

In [14]:
def build_estimator(tf_transform_dir, config, hidden_units=None):
    """Build an estimator for predicting the fare value of New York taxi riders.

    Args:
    tf_transform_dir: directory in which the tf-transform model was written
      during the preprocessing step.
    config: tf.contrib.learn.RunConfig defining the runtime environment for the
      estimator (including model_dir).
    hidden_units: [int], the layer sizes of the DNN (input layer first)

    Returns:
    Resulting DNNLinearCombinedClassifier.
    """
    metadata_dir = os.path.join(tf_transform_dir,
                              transform_fn_io.TRANSFORMED_METADATA_DIR)
    transformed_metadata = metadata_io.read_metadata(metadata_dir)
    transformed_feature_spec = transformed_metadata.schema.as_feature_spec()

    transformed_feature_spec.pop(transformed_name(LABEL_KEY))

    real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in transformed_names(DENSE_FLOAT_FEATURE_KEYS)
    ]
    categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=VOCAB_SIZE + OOV_SIZE, default_value=0)
      for key in transformed_names(VOCAB_FEATURE_KEYS)
    ]
    categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=FEATURE_BUCKET_COUNT, default_value=0)
      for key in transformed_names(BUCKET_FEATURE_KEYS)
    ]
    categorical_columns += [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=num_buckets, default_value=0)
      for key, num_buckets in zip(
          transformed_names(CATEGORICAL_FEATURE_KEYS),  #
          MAX_CATEGORICAL_FEATURE_VALUES)
    ]
    return tf.estimator.DNNLinearCombinedRegressor(
      config=config,
      dnn_feature_columns=real_valued_columns,
      dnn_hidden_units=hidden_units or [100, 70, 50, 25])

## Train, Evaluate, and Export our model

In [15]:
def example_serving_receiver_fn(tf_transform_dir, schema):
    """Build the serving in inputs.

    Args:
    tf_transform_dir: directory in which the tf-transform model was written
      during the preprocessing step.
    schema: the schema of the input data.

    Returns:
    Tensorflow graph which parses examples, applying tf-transform to them.
    """
    raw_feature_spec = get_raw_feature_spec(schema)
    raw_feature_spec.pop(LABEL_KEY)

    raw_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
      raw_feature_spec, default_batch_size=None)
    serving_input_receiver = raw_input_fn()

    _, transformed_features = (
      saved_transform_io.partially_apply_saved_transform(
          os.path.join(tf_transform_dir, transform_fn_io.TRANSFORM_FN_DIR),
          serving_input_receiver.features))

    return tf.estimator.export.ServingInputReceiver(
      transformed_features, serving_input_receiver.receiver_tensors)

In [16]:
def train_and_maybe_evaluate(hparams):
    """Run the training and evaluate using the high level API.

    Args:
    hparams: Holds hyperparameters used to train the model as name/value pairs.

    Returns:
    The estimator that was used for training (and maybe eval)
    """
    schema = read_schema(hparams.schema_file)

    train_input = lambda: input_fn(
      hparams.train_files,
      hparams.tf_transform_dir,
      batch_size=TRAIN_BATCH_SIZE
    )

    eval_input = lambda: input_fn(
      hparams.eval_files,
      hparams.tf_transform_dir,
      batch_size=EVAL_BATCH_SIZE
    )

    train_spec = tf.estimator.TrainSpec(
      train_input, max_steps=hparams.train_steps)

    serving_receiver_fn = lambda: example_serving_receiver_fn(
      hparams.tf_transform_dir, schema)

    exporter = tf.estimator.FinalExporter('nyc-taxi', serving_receiver_fn)
    eval_spec = tf.estimator.EvalSpec(
      eval_input,
      steps=hparams.eval_steps,
      exporters=[exporter],
      name='nyc-taxi-eval')

    run_config = tf.estimator.RunConfig(
      save_checkpoints_steps=999, keep_checkpoint_max=1)

    serving_model_dir = os.path.join(hparams.output_dir, SERVING_MODEL_DIR)
    run_config = run_config.replace(model_dir=serving_model_dir)

    estimator = build_estimator(
      hparams.tf_transform_dir,

      # Construct layers sizes with exponetial decay
      hidden_units=[
          max(2, int(FIRST_DNN_LAYER_SIZE * DNN_DECAY_FACTOR**i))
          for i in range(NUM_DNN_LAYERS)
      ],
      config=run_config)
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
    return estimator

In [17]:
# Transform training data
transform_data(input_handle=TRAIN_DATA,
                          outfile_prefix=TFT_TRAIN_FILE_PREFIX, 
                          working_dir=get_tft_train_output_dir(0),
                          schema_file=get_schema_file(),
                          pipeline_args=['--runner=DirectRunner'])
print('Done')

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done


In [18]:
# Compute stats over transformed training data.
TRANSFORMED_TRAIN_DATA = os.path.join(get_tft_train_output_dir(0), TFT_TRAIN_FILE_PREFIX + "*") 
transformed_train_stats = tfdv.generate_statistics_from_tfrecord(data_location=TRANSFORMED_TRAIN_DATA)

In [19]:
# Transform eval data
transform_data(input_handle=EVAL_DATA,
                          outfile_prefix=TFT_EVAL_FILE_PREFIX, 
                          working_dir=get_tft_eval_output_dir(0),
                          schema_file=get_schema_file(),
                          pipeline_args=['--runner=DirectRunner'])
print('Done')

INFO:tensorflow:Assets added to graph.


INFO:tensorflow:Assets added to graph.


INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /Users/arm/code/tfx/tensorflow-extended/notebooks/tft_eval/run_0/tftransform_tmp/92cf9e86f96f49a1ae7d2a59f4e5b067/saved_model.pb


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Done


In [20]:
def run_experiment(hparams):
    """Run the training and evaluate using the high level API"""

    # train and evaluate the model as usual
    estimator = train_and_maybe_evaluate(hparams)
    # export TFMA's sepcial EvalSavedModel
    eval_model_dir = os.path.join(hparams.output_dir, EVAL_MODEL_DIR)
    receiver_fn = lambda: eval_input_receiver_fn(hparams.tf_transform_dir)

    tfma.export.export_eval_savedmodel(
        estimator=estimator,
        export_dir_base=eval_model_dir,
        eval_input_receiver_fn=receiver_fn)
    
def eval_input_receiver_fn(working_dir):
    # extract feature spec from the schema
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec

    serialized_tf_example = tf.placeholder(
        dtype=tf.string, shape=[None], name='input_example_tensor')

    # deserialize our examples using the raw schema
    features = tf.parse_example(serialized_tf_example, raw_feature_spec)

    # now that we have our raw examples, we must process them through tft
    _, transformed_features = (
        saved_transform_io.partially_apply_saved_transform(
            os.path.join(working_dir, transform_fn_io.TRANSFORM_FN_DIR),
            features))

    # key MUST be 'examples'.
    print("serialized_tf_example: ", serialized_tf_example)

    receiver_tensors = {'examples': serialized_tf_example}
    print("\n\n receiver_tensors: ", receiver_tensors)
    # NOTE: Model is driven by transformed features (since training works on the
    # materialized output of TFT, but slicing will happen on raw features.
    features.update(transformed_features)
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=receiver_tensors,
        labels=transformed_features[transformed_name(LABEL_KEY)])

print('Done')

Done


In [21]:
def run_local_experiment(tft_run_id, tf_run_id, num_layers, first_layer_size, scale_factor):
    """Helper method to train and export the model for TFMA
    
    The caller specifies the input and output directory by providing run ids. The optional parameters
    allows the user to change the modelfor time series view.
    
    Args:
      tft_run_id: The run id for the preprocessing. Identifies the folder containing training data.
      tf_run_id: The run for this training run. Identify where the exported model will be written to.
      num_layers: The number of layers used by the hiden layer.
      first_layer_size: The size of the first hidden layer.
      scale_factor: The scale factor between each layer in in hidden layers.
    """
    print("train_files: ", os.path.join(get_tft_train_output_dir(tft_run_id), TFT_TRAIN_FILE_PREFIX + '-00000-of-*'))
    print("eval_files: ", os.path.join(get_tft_eval_output_dir(tft_run_id), TFT_EVAL_FILE_PREFIX + '-00000-of-*'))
    print("get_schema_file: ", get_schema_file())
    hparams = tf.contrib.training.HParams(
        # Inputs: are tf-transformed materialized features        
        train_files=os.path.join(get_tft_train_output_dir(tft_run_id), TFT_TRAIN_FILE_PREFIX + '-00000-of-*'),
        eval_files=os.path.join(get_tft_eval_output_dir(tft_run_id), TFT_EVAL_FILE_PREFIX + '-00000-of-*'),
        schema_file=get_schema_file(),
        # Output: dir for trained model
        job_dir=get_tf_output_dir(tf_run_id),
        tf_transform_dir=get_tft_train_output_dir(tft_run_id),
        
        # Output: dir for both the serving model and eval_model which will go into tfma
        # evaluation
        output_dir=get_tf_output_dir(tf_run_id),
        train_steps=10000,
        eval_steps=5000,
        num_layers=num_layers,
        first_layer_size=first_layer_size,
        scale_factor=scale_factor,
        num_epochs=None,
        train_batch_size=40,
        eval_batch_size=40)

    run_experiment(hparams)

print('Done')

Done


In [22]:
run_local_experiment(tft_run_id=0,
                     tf_run_id=0,
                     num_layers=4,
                     first_layer_size=100,
                     scale_factor=0.7)
print('Done')

train_files:  /Users/arm/code/tfx/tensorflow-extended/notebooks/tft_train/run_0/train_transformed-00000-of-*
eval_files:  /Users/arm/code/tfx/tensorflow-extended/notebooks/tft_eval/run_0/eval_transformed-00000-of-*
get_schema_file:  /Users/arm/code/tfx/tensorflow-extended/notebooks/schema.pbtxt
INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c37255b10>, '_model_dir': '/Users/arm/code/tfx/tensorflow-extended/notebooks/tf/run_0/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute

INFO:tensorflow:Using config: {'_save_checkpoints_secs': None, '_num_ps_replicas': 0, '_keep_checkpoint_max': 1, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1c37255b10>, '_model_dir': '/Users/arm/code/tfx/tensorflow-extended/notebooks/tf/run_0/serving_model_dir', '_protocol': None, '_save_checkpoints_steps': 999, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_device_fn': None, '_experimental_distribute': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_evaluation_master': '', '_eval_distribute': None, '_train_distribute': None, '_master': ''}


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

INFO:tensorflow:No assets to write.


INFO:tensorflow:No assets to write.


INFO:tensorflow:SavedModel written to: /Users/arm/code/tfx/tensorflow-extended/notebooks/tf/run_0/eval_model_dir/temp-1551254312/saved_model.pb


INFO:tensorflow:SavedModel written to: /Users/arm/code/tfx/tensorflow-extended/notebooks/tf/run_0/eval_model_dir/temp-1551254312/saved_model.pb


Done


## Summary

+ We used `tf.Transform` to preprocess a dataset and train a model with the cleaned and transformed data

+ We created an input function that we can be used when we deploy our trained model in a production environment to perform inference. As a result, we will avoid any issues pertaining to data skew since the same code is used during training & inference.  

+ We created an Apache Beam transform to perform the transformation that we needed for cleaing the data, and wrapped our data in TensorFlow FeatureColumns.